In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


# !mkdir -p Drive
# !google-drive-ocamlfuse Drive
!pip install transformers
!pip install tqdm
!pip3 install torch===1.3.1 torchvision===0.4.2 -f https://download.pytorch.org/whl/torch_stable.html

In [0]:
import math
import torch
import datetime
import operator
from tqdm import tqdm
from transformers import *
from google.colab import files
from torch.utils.data import DataLoader
from IPython.display import clear_output

In [0]:
!git clone https://gitlab.com/morrisChen/irhw5.git
filepath = "irhw5/"

def getTrainQueryList():
    queryListDoc = open(filepath+"train/query_list.txt",'r')
    queryList=[]
    for query in tqdm(queryListDoc):
        queryList.append(query.strip())
    queryListDoc.close()
    return queryList

def getTrainQueries(queryList):
    queries=[]
    for query in tqdm(range(len(queryList))):
        queryQuery=open(filepath+"train/query/"+queryList[query],'r')
        queries.append(queryQuery.read())
        queryQuery.close()
    return queries

def getTrainNeg(queryList):
    negDoc = open(filepath+"train/Neg.txt",'r')
    negs = {}
    for query in queryList:
        negs[query]=[]
    for neg in tqdm(negDoc):
        neg = neg.strip().split(' ')
        negs[neg[0]].append(neg[1])
    negDoc.close()
    return negs

def getTrainPos(queryList):
    posDoc = open(filepath+"train/Pos.txt",'r')
    poses = {}
    for query in queryList:
        poses[query]=[]
    for pos in tqdm(posDoc):
        pos = pos.strip().split(' ')
        poses[pos[0]].append(pos[1])
    posDoc.close()
    return poses

def getTestQueryList():
    queryListDoc = open(filepath+"test/query_list.txt",'r')
    queryList=[]
    for query in tqdm(queryListDoc):
        queryList.append(query.strip())
    queryListDoc.close()
    return queryList

def getTestQueries(queryList):
    queries=[]
    for query in tqdm(range(len(queryList))):
        queryQuery=open(filepath+"test/query/"+queryList[query],'r')
        queries.append(queryQuery.read())
        queryQuery.close()
    return queries
        
def getDocList():
    #docListDoc = open(filepath+"fileLessThan500Char.txt",'r')
    docListDoc = open(filepath+"doc/hw5.txt",'r')
    docList=[]
    for doc in tqdm(docListDoc):
        docList.append(doc.strip())
    docListDoc.close()
    return docList

def getDocs(docList):
    docs={}
    for doc in tqdm(range(len(docList))):
        docDoc=open(filepath+"doc/"+docList[doc],'r')
        docs[docList[doc]]=docDoc.read()
        docDoc.close()
    return docs
    
trainQueryList = getTrainQueryList()
trainQueries = getTrainQueries(trainQueryList)
trainQueriesNeg = getTrainNeg(trainQueryList)
trainQueriesPos = getTrainPos(trainQueryList)
testQueryList = getTestQueryList()
testQueries = getTestQueries(testQueryList)
docList = getDocList()
docs = getDocs(docList)

In [0]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
def getOneStream(text_a,text_b):
    tokens_tensor = tokenizer.encode(text_b, add_special_tokens=False)
    clear_output(wait=True)
    separate = math.ceil(len(tokens_tensor)/499)
    tokens_tensors = []
    for i in range(separate):
        query = torch.tensor(tokenizer.encode(text_a))
        doc = torch.tensor(tokens_tensor[i*499:(i+1)*499])
        sep = torch.tensor([102])
        tokens_tensors.append(torch.cat((query, doc, sep), 0))
        #tokens_tensors.append(torch.tensor([tokenizer.encode(text_a + "[SEP]" + tokenizer.decode(tokens_tensor[i*499:(i+1)*499]))]))
    return tokens_tensors

# def getOneStream(text_a,text_b):
#     tokens_tensors = []
#     tokens_tensor = tokenizer.encode(text_a + "[SEP]" + text_b)
#     return torch.tensor([tokens_tensor])

def getTrainData(qidx):
    tokens_tensors=[]
    label_tensors=[]
    fileNo=[]
    text_a = trainQueries[qidx]
    for didx in range(len(trainQueriesNeg[trainQueryList[qidx]])):
        text_b = docs[trainQueriesNeg[trainQueryList[qidx]][didx]]
        label_tensor = torch.tensor([0.])
        tokens_tensor_stream = getOneStream(text_a,text_b)
        for tensor in range(len(tokens_tensor_stream)):
            tokens_tensors.append(tokens_tensor_stream[tensor])
            label_tensors.append(label_tensor)

    for didx in range(len(trainQueriesPos[trainQueryList[qidx]])):
        text_b = docs[trainQueriesPos[trainQueryList[qidx]][didx]]
        label_tensor = torch.tensor([1.])
        tokens_tensor_stream = getOneStream(text_a,text_b)
        for tensor in range(len(tokens_tensor_stream)):
            tokens_tensors.append(tokens_tensor_stream[tensor])
            label_tensors.append(label_tensor)
        
    return (tokens_tensors, label_tensors)

# #15940 seperate to streams, each stream has BATCH doc
# BATCH=400
# STREAM=math.ceil(len(docList)/BATCH)
# def getTestData(qidx,d_batch):
#     fileName=[]
#     tokens_tensors=[]
#     text_a = testQueries[qidx]
#     startIndex = d_batch*BATCH
#     endIndex = (d_batch+1)*BATCH if (d_batch+1)*BATCH < len(docList) else len(docList)
#     for didx in range(startIndex,endIndex):
#         text_b = docs[didx]
#         tokens_tensor = getOneStream(text_a,text_b)
#         if(len(tokens_tensor[0])<510):
#           clear_output(wait=True)
#           tokens_tensors.append(tokens_tensor)
#           fileName.append(docList[didx])
#     return (tokens_tensors,fileName,startIndex)

def getTestData(qidx):
    tokens_tensors=[]
    fileNo=[]
    text_a = testQueries[qidx]
    for didx in range(len(docList)):
        text_b = docs[docList[didx]]
        tokens_tensor_stream = getOneStream(text_a,text_b)
        tokens_tensors.append(tokens_tensor_stream)
    return tokens_tensors

In [0]:
class SimpleBertIR(torch.nn.Module):
    def __init__(self):
        super(SimpleBertIR,self).__init__()
        self.bert=BertModel.from_pretrained('bert-base-uncased')
        self.Out_FC=torch.nn.Linear(768,1)
        
    def forward(self,_input_ids):
        outputs=self.bert(_input_ids.squeeze(1))
        CLS_representation=outputs[0][0][0]
        Pred_out=self.Out_FC(CLS_representation)
        return Pred_out
    

CUDA = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=SimpleBertIR().to(CUDA)

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
MSEloss=torch.nn.MSELoss()

model.train()
#for qidx in tqdm(range(10)):
for qidx in tqdm(range(len(trainQueries))):
    ##with torch.no_grad():
    tokens_tensors, label_tensors = getTrainData(qidx)
    for didx in range(len(tokens_tensors)):
        tokens_tensor = tokens_tensors[didx]
        label_tensor = label_tensors[didx]
        print(tokens_tensor)
        model_out=model(tokens_tensor.to(CUDA))
        output=MSEloss(model_out,label_tensor.to(CUDA))
        optimizer.zero_grad()
        output.backward()
        optimizer.step()
torch.save(model, filepath+"model.pt")

In [0]:
modelload = torch.load(filepath+"model.pt")

queryResult={}
result={}
with torch.no_grad():
    for qidx in tqdm(range(len(testQueries))):
        tokens_tensors,fileNo = getTestData(qidx)
        for didx in range(len(tokens_tensors)):
            tokens_tensor_streams = tokens_tensors[didx]
            for stream in range(len(tokens_tensor_streams)):
                result[docList[didx]] += modelload(tokens_tensor.to(CUDA))/len(tokens_tensor_streams)
            
        queryResult[testQueryList[qidx]] = sorted(result.items(), key=operator.itemgetter(1),reverse=True)[:100]

In [0]:
fileName = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")+"answer.txt"
file= open(filepath+fileName,"w+")
file.write("Query,RetrievedDocuments\n")
for query in queryResult:
    file.write("%s," % query)
    for doc in range(len(queryResult[query])):
        file.write(" %s" % queryResult[query][doc][0])
    file.write("\n")
file.close()
files.download(filepath+fileName)